In [75]:
import pandas as pd 
import os

In [76]:
folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [77]:
df = pd.read_csv('vehicles_df.csv')
df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [78]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66408 entries, 0 to 66407
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          66408 non-null  int64  
 1   Public Vehicle Number               66408 non-null  int64  
 2   Status                              66408 non-null  object 
 3   Vehicle Make                        58740 non-null  object 
 4   Vehicle Model                       58556 non-null  object 
 5   Vehicle Model Year                  58640 non-null  float64
 6   Vehicle Color                       58464 non-null  object 
 7   Vehicle Fuel Source                 66408 non-null  object 
 8   Wheelchair Accessible               66408 non-null  object 
 9   Company Name                        66408 non-null  object 
 10  Address                             59264 non-null  object 
 11  City                                59264

In [79]:
print(df.head())

   Unnamed: 0  Public Vehicle Number     Status Vehicle Make   Vehicle Model  \
0          56                   4595  VIOLATION        DODGE         CARAVAN   
1         129                   4540  VIOLATION        DODGE   GRAND CARAVAN   
2         148                   1759  VIOLATION        DODGE         CARAVAN   
3         248                   6897    REVOKED        DODGE   GRAND CARAVAN   
4         252                   5774  VIOLATION         FORD  CROWN VICTORIA   

   Vehicle Model Year Vehicle Color Vehicle Fuel Source Wheelchair Accessible  \
0              2017.0         WHITE           Flex Fuel                     N   
1              2017.0         WHITE           Flex Fuel                     Y   
2              2013.0         WHITE           Flex Fuel                     N   
3              2013.0         WHITE           Flex Fuel                     N   
4              2011.0         WHITE           Flex Fuel                     N   

       Company Name             

In [80]:
print(df.describe())

         Unnamed: 0  Public Vehicle Number  Vehicle Model Year      ZIP Code
count  66408.000000           6.640800e+04        58640.000000  59264.000000
mean    8300.500000           3.854566e+04         2012.669714  60612.519573
std     4792.620661           1.372915e+05           83.620667    104.288240
min        0.000000           0.000000e+00            0.000000  60004.000000
25%     4150.000000           3.633000e+03         2013.000000  60619.000000
50%     8300.500000           6.991500e+03         2016.000000  60631.000000
75%    12451.000000           1.684100e+04         2021.000000  60646.000000
max    16601.000000           6.902206e+06         2025.000000  61111.000000


In [81]:
df_cleaned = df.drop_duplicates(subset= 'Record ID', keep='first')

In [82]:
df_cleaned.isna().sum()

Unnamed: 0                               0
Public Vehicle Number                    0
Status                                   0
Vehicle Make                          1816
Vehicle Model                         1859
Vehicle Model Year                    1841
Vehicle Color                         1883
Vehicle Fuel Source                      0
Wheelchair Accessible                    0
Company Name                             0
Address                               1694
City                                  1694
State                                 1694
ZIP Code                              1694
Taxi Affiliation                      8710
Taxi Medallion License Management     8736
Record ID                                0
dtype: int64

In [83]:
columnCheck = ['Vehicle Make', 'Vehicle Model', 'Vehicle Model Year', 'Vehicle Color', 'Address','City', 'State', 'ZIP Code']

dfCleaned = df_cleaned.dropna(subset=columnCheck, how='all')

print(dfCleaned.isnull().sum())

Unnamed: 0                               0
Public Vehicle Number                    0
Status                                   0
Vehicle Make                           128
Vehicle Model                          171
Vehicle Model Year                     153
Vehicle Color                          195
Vehicle Fuel Source                      0
Wheelchair Accessible                    0
Company Name                             0
Address                                  6
City                                     6
State                                    6
ZIP Code                                 6
Taxi Affiliation                      7022
Taxi Medallion License Management     7048
Record ID                                0
dtype: int64


Remove records with a vehicle model year earlier than 1900, as no vehicles were manufactured before that year.

In [84]:
dfFltered = dfCleaned[dfCleaned['Vehicle Model Year']>=1900]

In [85]:
dfFltered.isna().sum()
dfFltered.shape

(13803, 17)

In [86]:
dfFltered.fillna("unknown", inplace=True)

/tmp/ipykernel_6001/267990826.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfFltered.fillna("unknown", inplace=True)
/tmp/ipykernel_6001/267990826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFltered.fillna("unknown", inplace=True)


In [87]:
# add new column name vehical type
dfFltered['Vehicle Type'] = dfFltered['Record ID'].str.extract(r'(\D+)$')[0]

/tmp/ipykernel_6001/3445207472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFltered['Vehicle Type'] = dfFltered['Record ID'].str.extract(r'(\D+)$')[0]


In [88]:
dfFltered.head()

,Unnamed: 0,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,Address,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
0,56,4595,VIOLATION,DODGE,CARAVAN,2017.0,WHITE,Flex Fuel,N,AMC CAB CORP.,3800 N. MILWAUKEE AVE.,CHICAGO,IL,60641.0,DIS-AFFILIATED,Owner Manager,4595Taxi,Taxi
1,129,4540,VIOLATION,DODGE,GRAND CARAVAN,2017.0,WHITE,Flex Fuel,Y,ELSTON FOUR LLC,3800 N. MILWAUKEE AVE.,CHICAGO,IL,60641.0,DIS-AFFILIATED,Owner Manager,4540Taxi,Taxi
2,148,1759,VIOLATION,DODGE,CARAVAN,2013.0,WHITE,Flex Fuel,N,1759 INC.,2601 W. PETERSON AVE.,CHICAGO,IL,60659.0,DIS-AFFILIATED,Owner Manager,1759Taxi,Taxi
3,248,6897,REVOKED,DODGE,GRAND CARAVAN,2013.0,WHITE,Flex Fuel,N,SHAZIF CAB CORP.,9696 W. FOSTER AVE.,CHICAGO,IL,60656.0,DIS-AFFILIATED,Owner Manager,6897Taxi,Taxi
4,252,5774,VIOLATION,FORD,CROWN VICTORIA,2011.0,WHITE,Flex Fuel,N,THOMAS A. ZUMMO,1756 W. CULLERTON ST.,CHICAGO,IL,60608.0,DIS-AFFILIATED,Owner Manager,5774Taxi,Taxi


In [89]:
# Drop the columns “Address” and “Public Vehicle Number”.
df_v1 = dfFltered.drop(columns=['Address', 'Public Vehicle Number'])


In [90]:
df_v1.head()

,Unnamed: 0,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
0,56,VIOLATION,DODGE,CARAVAN,2017.0,WHITE,Flex Fuel,N,AMC CAB CORP.,CHICAGO,IL,60641.0,DIS-AFFILIATED,Owner Manager,4595Taxi,Taxi
1,129,VIOLATION,DODGE,GRAND CARAVAN,2017.0,WHITE,Flex Fuel,Y,ELSTON FOUR LLC,CHICAGO,IL,60641.0,DIS-AFFILIATED,Owner Manager,4540Taxi,Taxi
2,148,VIOLATION,DODGE,CARAVAN,2013.0,WHITE,Flex Fuel,N,1759 INC.,CHICAGO,IL,60659.0,DIS-AFFILIATED,Owner Manager,1759Taxi,Taxi
3,248,REVOKED,DODGE,GRAND CARAVAN,2013.0,WHITE,Flex Fuel,N,SHAZIF CAB CORP.,CHICAGO,IL,60656.0,DIS-AFFILIATED,Owner Manager,6897Taxi,Taxi
4,252,VIOLATION,FORD,CROWN VICTORIA,2011.0,WHITE,Flex Fuel,N,THOMAS A. ZUMMO,CHICAGO,IL,60608.0,DIS-AFFILIATED,Owner Manager,5774Taxi,Taxi
